In [2]:
def compute_equiv_q(M1, M2, q):
    """
    Compute all sigma in GL_n(F_q) such that sigma * M1 * sigma^(-1) = M2
    
    Parameters:
    -----------
    M1, M2 : matrices (will be converted to F_q)
    q : prime power
    
    Returns:
    --------
    List of matrices sigma in GL_n(F_q) satisfying the equation
    """
    F = GF(q)
    n = M1.nrows()
    
    # Convert to matrices over F_q
    M1_q = matrix(F, M1)
    M2_q = matrix(F, M2)
    
    # Find all solutions
    solutions = []
    for sigma in GL(n, F):
        if sigma * M1_q * sigma^(-1) == M2_q:
            solutions.append(sigma)
    
    return solutions

In [3]:
def to_matrix(sigma):
    """
    Convert sigma to a matrix, handling both matrices and group elements.
    """
    if hasattr(sigma, 'matrix'):
        return sigma.matrix()
    else:
        return sigma

def is_permutation_matrix(sigma):
    """
    Check if sigma is a permutation matrix (entries in {0,1}, one 1 per row/col)
    """
    # Convert to matrix if needed
    mat = to_matrix(sigma)
    
    n = mat.nrows()
    F = mat.base_ring()
    
    # Check entries are 0 or 1
    for i in range(n):
        for j in range(n):
            if mat[i,j] not in [F(0), F(1)]:
                return False
    
    # Check exactly one 1 per row
    for i in range(n):
        if sum(mat[i,j] for j in range(n)) != F(1):
            return False
    
    # Check exactly one 1 per column
    for j in range(n):
        if sum(mat[i,j] for i in range(n)) != F(1):
            return False
    
    return True

In [4]:
# Example usage for n=3, q=2
F = GF(2)

# Path graph P_3: 1-2-3
M1 = matrix(F, [[0,1,0], [1,0,1], [0,1,0]])

# Same path, relabeled: 1-3-2
M2 = matrix(F, [[0,0,1], [0,0,1], [1,1,0]])

print("M1 (Path 1-2-3):")
print(M1)
print("\nM2 (Path 1-3-2):")
print(M2)

solutions = compute_equiv_q(M1, M2, 2)
print(f"\nFound {len(solutions)} solutions in GL_3(F_2)")
print(f"|GL_3(F_2)| = {GL(3, F).order()}")

perm_solutions = [s for s in solutions if is_permutation_matrix(s)]
print(f"\n{len(perm_solutions)} are permutation matrices")
print(f"{len(solutions) - len(perm_solutions)} are non-permutation matrices")

print("\nPermutation solutions:")
for i, sigma in enumerate(perm_solutions):
    print(f"\nPermutation {i+1}:")
    print(sigma)

print("\nNon-permutation solutions:")
for i, sigma in enumerate(solutions):
    if not is_permutation_matrix(sigma):
        print(f"\nNon-permutation {i+1}:")
        print(sigma)
        print(f"Determinant: {sigma.matrix().det()}")

# Verify one solution works
if solutions:
    sigma = solutions[0]
    result = sigma * M1 * sigma^(-1)
    print(f"\nVerification: sigma * M1 * sigma^(-1) == M2: {result == M2}")

M1 (Path 1-2-3):
[0 1 0]
[1 0 1]
[0 1 0]

M2 (Path 1-3-2):
[0 0 1]
[0 0 1]
[1 1 0]

Found 4 solutions in GL_3(F_2)
|GL_3(F_2)| = 168

2 are permutation matrices
2 are non-permutation matrices

Permutation solutions:

Permutation 1:
[0 0 1]
[1 0 0]
[0 1 0]

Permutation 2:
[1 0 0]
[0 0 1]
[0 1 0]

Non-permutation solutions:

Non-permutation 2:
[0 1 1]
[1 1 0]
[1 1 1]
Determinant: 1

Non-permutation 4:
[1 1 0]
[0 1 1]
[1 1 1]
Determinant: 1

Verification: sigma * M1 * sigma^(-1) == M2: True


In [5]:
def are_similar_fast(M1, M2, q):
    """
    Fast check if M1 and M2 are similar over F_q using canonical forms.
    Returns True/False without finding all solutions.
    """
    F = GF(q)
    M1_q = matrix(F, M1)
    M2_q = matrix(F, M2)
    
    # Check if they have the same rational canonical form
    # or characteristic polynomial as a quick test
    if M1_q.charpoly() != M2_q.charpoly():
        return False
    
    # For definitive answer, compare rational canonical forms
    try:
        # In Sage, this computes the rational canonical form
        rcf1 = M1_q.rational_form()
        rcf2 = M2_q.rational_form()
        return rcf1 == rcf2
    except:
        # Fallback: just use characteristic polynomial
        return M1_q.charpoly() == M2_q.charpoly()

In [6]:
are_similar_fast(M1,M2,2)

True

In [7]:
def compute_stabilizer(M, q, max_check=None):
    """
    Compute Stab_GL(M) = {sigma : sigma * M * sigma^(-1) = M}
    
    For large groups, set max_check to limit enumeration.
    """
    F = GF(q)
    n = M.nrows()
    M_q = matrix(F, M)
    
    stabilizer = []
    count = 0
    for g in GL(n, F):
        if max_check and count >= max_check:
            break
        sigma = g.matrix()
        if sigma * M_q * sigma^(-1) == M_q:
            stabilizer.append(sigma)
        count += 1
    
    return stabilizer

In [8]:
compute_stabilizer(M1,2)

[
[1 0 0]  [1 1 0]  [0 0 1]  [0 1 1]
[0 1 0]  [1 1 1]  [0 1 0]  [1 1 1]
[0 0 1], [0 1 1], [1 0 0], [1 1 0]
]

In [9]:
def find_one_solution(M1, M2, q):
    """
    If M1 and M2 are similar, find ONE solution sigma.
    This is much faster than enumerating all solutions.
    """
    F = GF(q)
    M1_q = matrix(F, M1)
    M2_q = matrix(F, M2)
    
    # Get rational canonical forms and transformation matrices
    rcf1, P1 = M1_q.rational_form(transformation=True)
    rcf2, P2 = M2_q.rational_form(transformation=True)
    
    if rcf1 != rcf2:
        return None  # Not similar
    
    # P1 * M1 * P1^(-1) = rcf = P2 * M2 * P2^(-1)
    # So: M2 = P2^(-1) * rcf * P2 = P2^(-1) * P1 * M1 * P1^(-1) * P2
    # Therefore: sigma = P2^(-1) * P1 satisfies sigma * M1 * sigma^(-1) = M2
    sigma = P2^(-1) * P1
    return sigma

In [10]:

# `rational_form()` does not provide the change-of-basis matrix.
try:
    find_one_solution(M1,M2,2)
except TypeError as e:
    print(e)


rational_form() got an unexpected keyword argument 'transformation'


In [11]:
def find_similarity_transform(M1, M2, q):
    """
    Find sigma such that sigma * M1 * sigma^(-1) = M2 using Jordan form.
    Returns None if M1 and M2 are not similar.
    
    Uses the Sage convention: P^(-1) * M * P = J, i.e., M = P * J * P^(-1)
    """
    F = GF(q)
    M1_q = matrix(F, M1)
    M2_q = matrix(F, M2)
    
    # Quick check with rational form
    if M1_q.rational_form(format='invariants') != M2_q.rational_form(format='invariants'):
        return None
    
    try:
        # Get Jordan forms with transformation matrices
        J1, P1 = M1_q.jordan_form(transformation=True, subdivide=False)
        J2, P2 = M2_q.jordan_form(transformation=True, subdivide=False)
        
        if J1 != J2:
            return None
        
        # Since M1 = P1 * J1 * P1^(-1) and M2 = P2 * J2 * P2^(-1)
        # and J1 = J2, we have:
        # M2 = P2 * J1 * P2^(-1) = P2 * P1^(-1) * M1 * P1 * P2^(-1)
        # So: sigma = P2 * P1^(-1) satisfies sigma * M1 * sigma^(-1) = M2
        
        sigma = P2 * P1^(-1)
        return sigma
            
    except Exception as e:
        print(f"Jordan form computation failed: {e}")
        return None

In [12]:
# Now we can efficiently compute all solutions!
def compute_all_solutions(M1, M2, q):
    """
    Efficiently compute all sigma in GL_n(F_q) with sigma * M1 * sigma^(-1) = M2.
    
    Strategy:
    1. Find one solution sigma0 using Jordan form
    2. Compute Stab(M1)
    3. All solutions are sigma0 * Stab(M1)
    """
    F = GF(q)
    M1_q = matrix(F, M1)
    M2_q = matrix(F, M2)
    
    # Find one solution
    sigma0 = find_similarity_transform(M1, M2, q)
    
    if sigma0 is None:
        return []
    
    # Verify it works
    assert sigma0 * M1_q * sigma0.inverse() == M2_q, "Solution verification failed!"
    
    # Compute stabilizer of M1
    stab = compute_stabilizer(M1, q)
    
    # All solutions are sigma0 * s for s in Stab(M1)
    all_solutions = [sigma0 * s for s in stab]
    
    return all_solutions

In [44]:
# Test on your original example
q=2
F = GF(q)
M1 = matrix(F, [[0,1,0], [1,0,1], [0,1,0]])
M2 = matrix(F, [[0,0,1], [0,0,1], [1,1,0]])

print("q=",q)
print("M1.charpoly(): ", M1.charpoly().factor())
print("M2.charpoly(): ", M2.charpoly().factor())
form1 = M1.jordan_form(transformation=True, subdivide=False)
form2 = M2.jordan_form(transformation=True, subdivide=False)
print("M1 Jordan canonical form: ", form1)
print("M2 Jordan canonical form: ", form2)

print("Finding one solution using Jordan form:")
sigma0 = find_similarity_transform(M1, M2, q)
if sigma0:
    print("Found sigma0:")
    print(sigma0)
    print("Verification:", sigma0 * M1 * sigma0^(-1) == M2)
    print("Is permutation:", is_permutation_matrix(sigma0))
    
    print("\nComputing all solutions:")
    all_sols = compute_all_solutions(M1, M2, q)
    perm_count = sum(1 for s in all_sols if is_permutation_matrix(s))
    
    print(f"Total solutions: {len(all_sols)}")
    print(f"Permutation solutions: {perm_count}")
    print(f"Non-permutation solutions: {len(all_sols) - perm_count}")
else:
    print("No solution found (matrices not similar)")

q= 2
M1.charpoly():  x^3
M2.charpoly():  x^3
M1 Jordan canonical form:  ([0 1 0]
[0 0 1]
[0 0 0], [1 0 1]
[0 1 0]
[1 0 0])
M2 Jordan canonical form:  ([0 1 0]
[0 0 1]
[0 0 0], [1 0 1]
[1 0 0]
[0 1 0])
Finding one solution using Jordan form:
Found sigma0:
[1 0 0]
[0 0 1]
[0 1 0]
Verification: True
Is permutation: True

Computing all solutions:
Total solutions: 4
Permutation solutions: 2
Non-permutation solutions: 2


In [5]:
q=3**2
F = GF(q)
M1 = matrix(F, [[0,1,0], [1,0,1], [0,1,0]])

print("q=",q)
print("M1.charpoly(): ", M1.charpoly().factor())
try:
    form1 = M1.jordan_form(transformation=True, subdivide=False)
except TypeError as e:
    print(e)

q= 9
M1.charpoly():  x * (x + z2 + 2) * (x + 2*z2 + 1)
'NoneType' object is not iterable
